# 21장. 훈련된 모델 저장과 복원

이 노트북을 주피터 노트북 뷰어(nbviewer.jupyter.org)로 보거나 구글 코랩(colab.research.google.com)에서 실행할 수 있습니다.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/machine-learning-with-python-cookbook/blob/master/21.ipynb"><img src="https://jupyter.org/assets/main-logo.svg" width="28" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/machine-learning-with-python-cookbook/blob/master/21.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

In [ ]:
# 코랩에서 실행하는 경우 다음 주석을 제거하고 실행하세요.
#!pip install numpy==1.16.2

## 21.1 사이킷런 모델을 저장하고 복원하기

In [1]:
# 라이브러리를 임포트합니다.
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
import joblib

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 결정 트리 분류기 객체를 만듭니다.
classifer = RandomForestClassifier()

# 모델을 훈련합니다.
model = classifer.fit(features, target)

# 모델을 피클 파일로 저장합니다.
joblib.dump(model, "model.pkl")

/home/haesun/anaconda3/envs/ml-cookbook/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


['model.pkl']

In [2]:
# 파일에서 모델을 복원합니다.
classifer = joblib.load("model.pkl")

In [3]:
# 새로운 샘플을 만듭니다.
new_observation = [[ 5.2,  3.2,  1.1,  0.1]]

# 샘플의 클래스를 예측합니다.
classifer.predict(new_observation)

array([0])

In [4]:
# 라이브러리를 임포트합니다.
import sklearn

# 사이킷런 버전을 구합니다.
scikit_version = sklearn.__version__

# 모델을 피클 파일로 저장합니다.
joblib.dump(model, "model_{version}.pkl".format(version=scikit_version))

['model_0.21.2.pkl']

## 21.2 케라스 모델을 저장하고 복원하기

In [5]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [6]:
# 라이브러리를 임포트합니다.
import numpy as np
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from keras.models import load_model

# 랜덤 시드를 지정합니다.
np.random.seed(0)

# 원하는 특성 개수를 설정합니다.
number_of_features = 1000

# 영화 리뷰 데이터와 타깃 벡터를 로드합니다.
(train_data, train_target), (test_data, test_target) = imdb.load_data(
    num_words=number_of_features)

# 영화 리뷰 데이터를 원-핫 인코딩된 특성 행렬로 변환합니다.
tokenizer = Tokenizer(num_words=number_of_features)
train_features = tokenizer.sequences_to_matrix(train_data, mode="binary")
test_features = tokenizer.sequences_to_matrix(test_data, mode="binary")

# 신경망을 모델을 만듭니다.
network = models.Sequential()

# ReLU 활성화 함수를 사용한 완전 연결 층을 추가합니다.
network.add(layers.Dense(units=16,
                         activation="relu",
                         input_shape=(number_of_features,)))

# 시그모이드 활성화 함수를 사용한 완전 연결 층을 추가합니다.
network.add(layers.Dense(units=1, activation="sigmoid"))

# 신경망 모델의 설정을 완료합니다.
network.compile(loss="binary_crossentropy", # 크로스 엔트로피
                optimizer="rmsprop", # 최적화 알고리즘
                metrics=["accuracy"]) # 정확도

# 신경망을 훈련합니다.
history = network.fit(train_features, # 특성
                      train_target, # 타깃 벡터
                      epochs=3, # 에포크 횟수
                      verbose=0, # 출력 없음
                      batch_size=100, # 배치 샘플 수
                      validation_data=(test_features, test_target)) # 테스트 데이터

# 신경망 모델을 저장합니다.
network.save("model.h5")

Using TensorFlow backend.


In [7]:
# 신경망 모델을 복원합니다.
network = load_model("model.h5")